In [4]:
import os
os.chdir("../")
import config.input
import numpy as np
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [5]:
# train_loader_fn, test_loader_fn = config.input.options["left_out_scale_mnist"]
# train_loader_fn, test_loader_fn = config.input.options["left_out_many_scale_biased_mnist"]
# train_loader_fn, test_loader_fn = config.input.options["left_out_many_scale_mnist"]
train_loader_fn, test_loader_fn = config.input.options["left_out_colored_mnist"] 
# train_loader_fn, test_loader_fn = config.input.options["left_out_varied_location_mnist"] 
# train_loader_fn, test_loader_fn = config.input.options["left_out_colored_location_mnist"]
train_loader_fn
np.random.seed(17)
random_indices = np.arange(10)
np.random.shuffle(random_indices)
        
args = {
    "batch_size": 1,
    "test_batch_size": 1,
    "use_cuda": False,
    "keep_pct": 3 , 
    "color_indices": random_indices,
    "example_pct": 1,
}

In [6]:
train_loader = train_loader_fn(args)
test_loader = test_loader_fn(args)

train_values = np.zeros(train_loader.dataset.combination_space_shape)
test_values = np.zeros(test_loader.dataset.combination_space_shape)

for _, (_, target) in enumerate(train_loader):
    train_values[target.data[0][0], target.data[0][1]] += 1
    
for _, (_, target) in enumerate(test_loader):
    test_values[target.data[0][0], target.data[0][1]] += 1

INIT
/Users/timhenry/Documents/mit/meng/../data/synth/LeftOutColoredMNIST/3
ROOT: ../data/synth/LeftOutColoredMNIST/3


RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
def display_data_hist(values):
    %matplotlib notebook

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.bar3d([x for x in range(values.shape[0]) for y in range(values.shape[1])], [y for x in range(values.shape[0]) for y in range(values.shape[1])], np.zeros(()), np.ones(()), np.ones(()), values.flatten(), zsort='average')

    plt.show()

In [ ]:
display_data_hist(train_values)

In [ ]:
display_data_hist(test_values)

In [ ]:
control_test_values = np.zeros(test_values.shape)
for i, j in test_loader.dataset.control:
    control_test_values[i, j] = test_values[i, j]
display_data_hist(control_test_values)

In [ ]:
held_out_test_values = np.zeros(test_values.shape)
for i, j in test_loader.dataset.held_out:
    held_out_test_values[i, j] = test_values[i, j]
display_data_hist(held_out_test_values)

In [ ]:
held_out_test_values = np.zeros(test_values.shape)
for i, j in test_loader.dataset.control_val:
    held_out_test_values[i, j] = test_values[i, j]
display_data_hist(held_out_test_values)

In [ ]:
def display_stimuli(loader):
    border_mask = np.ones((32, 32, 3)) * 255
    border_mask[1:-1, 1:-1, :] = 0
    big_border_mask = np.ones((34, 34, 3)) * 255
    big_border_mask[2:-2, 2:-2, :] = 0
    
    output = np.zeros((32 * loader.dataset.combination_space_shape[0], \
                       32 * loader.dataset.combination_space_shape[1], \
                       3))
    for _, (data, target) in enumerate(loader):
        label_1, label_2 = int(target.data[0][0]), int(target.data[0][1])
        offset_1, offset_2 = label_1 * 32, label_2 * 32
        output[offset_1:(offset_1 + 32), offset_2:(offset_2 + 32),:] = np.moveaxis(data[0].numpy(), 0, -1) * 255 + border_mask
        try:
            if offset_1-1 >= 0:
                output[offset_1-1, offset_2:offset_2+32] = 255
        except:
            pass
        try:
            output[offset_1+32, offset_2:offset_2+32] = 255
        except:
            pass
        try:
            if offset_2-1 >= 0:
                output[offset_1:offset_1+32, offset_2-1] = 255
        except:
            pass
        try:
            output[offset_1:offset_1+32, offset_2+32] = 255
        except:
            pass

    return output.astype("uint8")

In [ ]:
train_output = display_stimuli(train_loader)
    
train_img = Image.fromarray(train_output)
train_img

In [ ]:
test_output = display_stimuli(test_loader)
    
test_img = Image.fromarray(test_output)
test_img

In [ ]:
held_out_test_output = np.zeros(test_output.shape, dtype="uint8")
for i, j in test_loader.dataset.held_out:
    idx_1, idx_2 = i * 32, j * 32
    held_out_test_output[idx_1:(idx_1 + 32), idx_2:(idx_2 + 32), :] = test_output[idx_1:(idx_1 + 32), idx_2:(idx_2 + 32), :]
    offset_1, offset_2 = idx_1, idx_2
    output = held_out_test_output
    try:
        if offset_1-1 >= 0:
            output[offset_1-1, offset_2:offset_2+32] = 255
    except:
        pass
    try:
        output[offset_1+32, offset_2:offset_2+32] = 255
    except:
        pass
    try:
        if offset_2-1 >= 0:
            output[offset_1:offset_1+32, offset_2-1] = 255
    except:
        pass
    try:
        output[offset_1:offset_1+32, offset_2+32] = 255
    except:
        pass
    
    
held_out_test_img = Image.fromarray(held_out_test_output)
held_out_test_img

In [ ]:
control_test_output = np.zeros(test_output.shape, dtype="uint8")
for i, j in test_loader.dataset.control:
    idx_1, idx_2 = i * 32, j * 32
    control_test_output[idx_1:(idx_1 + 32), idx_2:(idx_2 + 32), :] = test_output[idx_1:(idx_1 + 32), idx_2:(idx_2 + 32), :]
    offset_1, offset_2 = idx_1, idx_2
    output = control_test_output
    try:
        if offset_1-1 >= 0:
            output[offset_1-1, offset_2:offset_2+32] = 255
    except:
        pass
    try:
        output[offset_1+32, offset_2:offset_2+32] = 255
    except:
        pass
    try:
        if offset_2-1 >= 0:
            output[offset_1:offset_1+32, offset_2-1] = 255
    except:
        pass
    try:
        output[offset_1:offset_1+32, offset_2+32] = 255
    except:
        pass
control_test_img = Image.fromarray(control_test_output)
control_test_img

In [ ]:
def get_pct_to_dict(pct):
    d = 9
    shapes = np.arange(d)
    colors = np.arange(d)
    locs = np.arange(d) 
    pct_to_dict = []
    for keep_pct in range(pct):
        pct_valid = {}
        for i in range(d):
            for j in range(d):
                shape = shapes[i]
                color = colors[(keep_pct + i + j) % d]
                loc = locs[(i + j) % d]
                if shape not in pct_valid:
                    pct_valid[shape] = {}
                if color not in pct_valid[shape]:
                    pct_valid[shape][color] = []
                pct_valid[shape][color].append(loc)
        pct_to_dict.append(pct_valid)
    return pct_to_dict
        
def get_combs(pct):
    combs = set()
    for dic in get_pct_to_dict(pct):
        for shape in dic:
            for color in dic[shape]:
                for loc in dic[shape][color]:
                    combs.add((shape, color, loc))
    return combs

In [ ]:
all_combs = get_combs(9)
train_combs = get_combs(8)
print("CONTROL")
print(list(get_combs(1)))

print("HELD OUT")
print(list(all_combs - train_combs))


In [ ]:
import random

shape_idx, col_idx, loc_idx = range(3)
space_shape = (9, 9, 9)

all_combs = set([(s, c, l) \
                 for s in range(space_shape[shape_idx]) \
                 for c in range(space_shape[col_idx]) \
                 for l in range(space_shape[loc_idx])
                ])
print(len(all_combs))

pct_to_combs = { num:[] for num in range(max(space_shape))}

for pct in range(max(space_shape)):
    shape_to_amt_needed = { num: space_shape[shape_idx] for num in range(space_shape[shape_idx]) }
    col_to_amt_needed = { num: space_shape[col_idx] for num in range(space_shape[col_idx]) }
    loc_to_amt_needed = { num: space_shape[loc_idx] for num in range(space_shape[loc_idx]) }
    
    combs_to_readd = set()
    while len(shape_to_amt_needed) + len(col_to_amt_needed) + len(loc_to_amt_needed) > 0:
        s, c, l = random.sample(all_combs, k=1)[0]
        all_combs.remove((s, c, l))
        assert (s, c, l) not in pct_to_combs[pct]
        if s in shape_to_amt_needed and \
           c in col_to_amt_needed and \
           l in loc_to_amt_needed and \
           :
            if shape_to_amt_needed[s] == 1:
                del shape_to_amt_needed[s]
            else:
                shape_to_amt_needed[s] -= 1

            if col_to_amt_needed[c] == 1:
                del col_to_amt_needed[c]
            else:
                col_to_amt_needed[c] -= 1

            if loc_to_amt_needed[l] == 1:
                del loc_to_amt_needed[l]
            else:
                loc_to_amt_needed[l] -= 1
            pct_to_combs[pct].append((s, c, l))
        else:
            combs_to_readd.add((s, c, l))
    all_combs.update(combs_to_readd)
    print(len(pct_to_combs[pct]))
    assert(len(pct_to_combs[pct]) == max(space_shape) ** 2)
    print(len(all_combs))
    assert(len(all_combs) == (max(space_shape) ** 3) - ((pct + 1) * (max(space_shape) ** 2)))

# print(pct_to_combs)
for key, val in pct_to_combs.items():
    print("PCT:", key, "/", max(space_shape))
    print(val)
    print("\n")

In [ ]:
for pct in pct_to_combs:
    print(pct, "/ 9")
    vals = {}
    for (s, c, l) in pct_to_combs[pct]:
        num = s * 100 + c * 10 + l
        if num in vals:
            vals[num] += 1
        else:
            vals[num] = 1
            
    x, y = [], []
    for key, val in vals.items():
        x.append(key)
        y.append(val)
        
    print(y)
    plt.plot(x, y)
    plt.show()
        

In [ ]:
pct_to_combs = {8: {(8, 3, 2), (1, 4, 0), (1, 7, 0), (0, 2, 7), (6, 5, 3), (3, 8, 6), (4, 1, 1), (5, 3, 2), (1, 8, 6), (0, 8, 8), (6, 6, 7), (4, 6, 5), (5, 8, 7), (7, 5, 2), (7, 3, 2), (1, 8, 8), (8, 2, 5), (2, 3, 0), (2, 4, 0), (3, 4, 3), (0, 1, 6), (6, 2, 4), (5, 7, 5), (5, 1, 1), (7, 0, 5), (3, 6, 4), (2, 2, 3), (0, 5, 8), (4, 0, 8), (8, 1, 2), (4, 4, 3), (0, 5, 4), (6, 2, 8), (8, 1, 6), (1, 1, 4), (4, 2, 3), (4, 4, 1), (6, 0, 1), (3, 0, 4), (2, 5, 8), (3, 6, 8), (0, 5, 0), (6, 5, 8), (3, 7, 7), (4, 7, 5), (8, 3, 1), (2, 0, 6), (0, 7, 7), (3, 1, 1), (1, 7, 5), (7, 4, 4), (8, 8, 4), (2, 6, 2), (4, 1, 0), (5, 3, 3), (1, 2, 5), (6, 6, 0), (7, 5, 5), (0, 8, 7), (7, 5, 3), (5, 8, 4), (5, 6, 1), (3, 4, 6), (8, 2, 2), (1, 0, 0), (7, 6, 7), (2, 3, 7), (4, 6, 8), (5, 7, 6), (1, 0, 4), (2, 3, 3), (8, 4, 3), (0, 3, 2), (8, 4, 1), (5, 7, 0), (6, 1, 7), (7, 0, 2), (6, 8, 5), (3, 0, 1), (7, 7, 6), (2, 2, 6)}, 7: {(5, 2, 7), (8, 0, 4), (2, 6, 7), (0, 2, 3), (5, 5, 5), (8, 8, 7), (1, 8, 2), (0, 7, 2), (7, 5, 6), (3, 2, 4), (8, 5, 7), (2, 5, 5), (3, 4, 5), (0, 4, 6), (7, 5, 0), (1, 5, 3), (7, 6, 6), (4, 3, 2), (5, 8, 1), (4, 3, 0), (5, 1, 7), (0, 0, 7), (2, 3, 2), (1, 1, 8), (6, 4, 0), (4, 5, 0), (2, 4, 4), (4, 5, 2), (6, 2, 0), (0, 6, 3), (5, 7, 1), (5, 4, 1), (8, 1, 0), (4, 2, 7), (3, 0, 0), (3, 0, 2), (6, 8, 6), (8, 1, 8), (2, 7, 3), (4, 7, 7), (1, 7, 3), (6, 7, 3), (8, 0, 1), (6, 0, 5), (1, 4, 1), (7, 1, 6), (6, 3, 8), (2, 5, 2), (3, 7, 3), (7, 3, 5), (1, 2, 3), (5, 8, 8), (3, 2, 5), (7, 3, 1), (6, 6, 6), (0, 8, 5), (1, 0, 2), (6, 6, 4), (7, 8, 7), (7, 0, 8), (0, 3, 8), (4, 3, 3), (8, 2, 4), (5, 1, 4), (4, 8, 8), (0, 6, 6), (5, 1, 0), (3, 3, 4), (2, 1, 6), (0, 0, 2), (3, 4, 8), (4, 4, 8), (1, 1, 1), (8, 4, 5), (1, 6, 1), (3, 6, 1), (6, 7, 4), (2, 8, 6), (8, 6, 4), (2, 7, 0), (7, 2, 5)}, 6: {(0, 5, 5), (3, 0, 3), (2, 2, 4), (5, 2, 1), (8, 3, 0), (1, 7, 2), (2, 7, 6), (0, 5, 3), (4, 2, 8), (3, 2, 2), (7, 1, 5), (2, 6, 3), (3, 7, 2), (1, 2, 4), (1, 5, 5), (6, 5, 1), (8, 8, 1), (5, 0, 0), (7, 3, 6), (1, 3, 7), (4, 1, 3), (5, 3, 6), (7, 6, 0), (6, 1, 8), (0, 4, 4), (3, 4, 1), (0, 6, 5), (6, 7, 6), (5, 6, 8), (2, 4, 2), (4, 0, 6), (3, 3, 7), (0, 0, 1), (3, 6, 2), (4, 2, 5), (5, 4, 7), (4, 8, 1), (8, 4, 4), (6, 7, 7), (2, 2, 5), (4, 7, 1), (7, 1, 0), (7, 4, 0), (4, 1, 4), (8, 3, 7), (3, 1, 3), (6, 5, 4), (8, 8, 6), (7, 1, 4), (1, 0, 8), (5, 0, 3), (2, 5, 6), (8, 8, 0), (2, 5, 4), (1, 8, 7), (6, 3, 0), (8, 5, 0), (1, 8, 5), (0, 6, 8), (6, 3, 6), (7, 7, 8), (2, 3, 1), (4, 0, 1), (7, 8, 3), (0, 0, 6), (0, 1, 7), (8, 2, 8), (5, 7, 2), (1, 6, 5), (0, 6, 2), (8, 4, 7), (7, 7, 0), (4, 8, 2), (3, 6, 3), (6, 0, 8), (5, 4, 4), (6, 8, 3), (2, 2, 2), (3, 5, 7), (1, 1, 5), (5, 4, 8)}, 5: {(1, 1, 7), (3, 1, 8), (2, 8, 2), (4, 7, 4), (3, 0, 7), (0, 7, 8), (5, 0, 6), (2, 8, 8), (3, 2, 0), (8, 8, 5), (1, 2, 2), (2, 6, 1), (5, 0, 2), (1, 8, 0), (7, 3, 4), (4, 6, 1), (0, 7, 0), (5, 3, 0), (7, 5, 4), (0, 4, 0), (1, 3, 5), (6, 3, 1), (8, 2, 1), (4, 6, 7), (8, 5, 3), (2, 3, 6), (8, 7, 6), (5, 1, 5), (5, 0, 8), (6, 4, 6), (0, 6, 7), (0, 6, 1), (1, 6, 4), (6, 4, 2), (8, 7, 0), (0, 3, 3), (6, 1, 6), (2, 8, 5), (5, 4, 5), (1, 6, 2), (6, 8, 2), (2, 2, 1), (7, 7, 7), (4, 7, 3), (3, 5, 4), (6, 7, 1), (7, 1, 2), (8, 0, 3), (1, 4, 3), (2, 0, 4), (3, 1, 5), (7, 4, 2), (4, 1, 6), (1, 5, 8), (5, 0, 5), (5, 5, 8), (8, 5, 4), (7, 4, 6), (0, 4, 3), (3, 8, 7), (4, 6, 2), (5, 8, 6), (7, 3, 3), (4, 6, 4), (6, 2, 7), (0, 8, 1), (2, 4, 1), (8, 2, 6), (6, 1, 3), (7, 0, 0), (1, 3, 8), (6, 7, 8), (3, 3, 8), (3, 5, 3), (0, 0, 0), (4, 2, 4), (4, 5, 7), (2, 2, 0), (7, 2, 7), (8, 1, 5), (3, 5, 5)}, 4: {(4, 7, 0), (8, 0, 6), (1, 4, 4), (2, 7, 2), (8, 6, 0), (2, 7, 4), (2, 0, 5), (0, 5, 1), (6, 0, 2), (8, 6, 2), (3, 1, 4), (6, 5, 7), (7, 3, 8), (2, 0, 1), (0, 4, 8), (3, 8, 2), (4, 1, 5), (2, 5, 3), (0, 1, 8), (0, 8, 6), (5, 6, 0), (7, 8, 4), (8, 5, 1), (5, 8, 5), (5, 3, 4), (3, 4, 7), (1, 3, 1), (5, 7, 3), (8, 4, 2), (1, 6, 6), (4, 8, 3), (2, 4, 6), (3, 6, 0), (7, 2, 8), (1, 1, 0), (7, 7, 5), (2, 1, 1), (8, 6, 5), (6, 8, 4), (5, 2, 6), (0, 2, 0), (6, 5, 6), (0, 2, 2), (6, 0, 7), (5, 5, 4), (1, 7, 7), (3, 2, 3), (1, 2, 1), (8, 3, 5), (5, 2, 8), (0, 7, 5), (7, 3, 7), (1, 3, 6), (5, 3, 7), (7, 5, 1), (6, 3, 4), (4, 1, 8), (3, 4, 0), (0, 1, 3), (4, 0, 3), (3, 3, 0), (8, 8, 8), (3, 4, 2), (8, 5, 8), (7, 0, 4), (6, 2, 3), (2, 4, 5), (7, 0, 6), (4, 5, 1), (3, 6, 7), (4, 0, 5), (0, 6, 0), (2, 4, 7), (6, 8, 1), (7, 7, 2), (4, 2, 6), (1, 1, 3), (1, 6, 3), (5, 1, 8), (4, 7, 2), (6, 8, 7)}, 3: {(7, 2, 3), (7, 1, 1), (3, 7, 4), (8, 0, 0), (8, 3, 4), (3, 1, 2), (6, 5, 5), (7, 4, 3), (2, 0, 3), (0, 7, 4), (1, 7, 4), (8, 6, 8), (8, 8, 3), (2, 5, 1), (6, 6, 1), (7, 4, 7), (1, 2, 6), (3, 7, 8), (4, 3, 6), (0, 8, 4), (0, 1, 2), (0, 8, 2), (7, 8, 0), (6, 4, 4), (3, 3, 5), (6, 1, 0), (7, 5, 8), (5, 1, 3), (2, 1, 5), (1, 6, 0), (7, 7, 3), (7, 0, 3), (0, 3, 7), (4, 8, 7), (0, 2, 8), (6, 7, 5), (3, 5, 6), (2, 8, 1), (2, 0, 8), (7, 2, 2), (0, 5, 6), (1, 1, 6), (1, 4, 5), (3, 7, 5), (6, 0, 3), (3, 1, 7), (5, 2, 2), (2, 6, 6), (2, 6, 4), (5, 0, 7), (5, 5, 6), (3, 2, 1), (8, 8, 2), (6, 3, 2), (8, 2, 0), (1, 5, 0), (4, 6, 6), (1, 3, 2), (4, 3, 5), (2, 6, 8), (5, 8, 0), (6, 4, 7), (4, 3, 1), (5, 6, 7), (6, 4, 1), (8, 7, 5), (4, 0, 7), (0, 0, 4), (2, 2, 8), (5, 4, 0), (8, 7, 1), (5, 1, 2), (4, 5, 3), (1, 6, 7), (4, 8, 0), (3, 5, 1), (4, 4, 6), (0, 3, 4), (1, 4, 8), (8, 0, 8), (5, 2, 5)}, 2: {(8, 1, 7), (4, 2, 2), (6, 7, 0), (7, 1, 3), (5, 5, 1), (6, 7, 2), (0, 2, 1), (8, 0, 2), (5, 5, 3), (5, 2, 3), (3, 8, 0), (8, 3, 6), (2, 6, 5), (0, 2, 5), (4, 7, 8), (1, 2, 0), (6, 0, 6), (5, 5, 7), (3, 8, 4), (8, 3, 8), (2, 3, 8), (4, 6, 3), (0, 4, 2), (1, 8, 4), (1, 3, 3), (4, 3, 4), (6, 3, 5), (3, 3, 1), (0, 1, 4), (4, 0, 2), (5, 7, 7), (7, 8, 2), (8, 7, 4), (4, 0, 0), (8, 7, 2), (0, 0, 5), (3, 6, 6), (7, 0, 7), (7, 8, 8), (7, 7, 1), (7, 0, 1), (4, 5, 6), (7, 2, 6), (8, 1, 4), (2, 2, 7), (8, 4, 8), (1, 4, 7), (5, 2, 0), (6, 8, 8), (7, 2, 0), (3, 5, 8), (6, 6, 8), (3, 0, 8), (2, 7, 7), (3, 8, 1), (2, 5, 0), (1, 8, 3), (6, 6, 2), (7, 5, 7), (2, 6, 0), (1, 8, 1), (1, 5, 4), (5, 0, 1), (0, 7, 1), (1, 5, 6), (4, 3, 7), (0, 4, 5), (5, 6, 3), (6, 4, 5), (0, 1, 5), (0, 6, 4), (2, 4, 3), (3, 3, 6), (3, 6, 5), (5, 4, 6), (2, 1, 2), (6, 1, 5), (2, 1, 0), (8, 1, 3), (4, 4, 4), (1, 4, 6)}, 1: {(1, 2, 8), (2, 8, 0), (4, 4, 0), (6, 0, 0), (3, 0, 5), (1, 4, 2), (3, 1, 6), (7, 1, 7), (3, 7, 6), (2, 0, 7), (6, 0, 4), (7, 4, 1), (0, 7, 6), (1, 7, 6), (5, 0, 4), (7, 6, 8), (4, 1, 7), (6, 6, 3), (3, 1, 0), (7, 4, 5), (3, 2, 6), (8, 5, 5), (1, 5, 7), (7, 6, 2), (7, 3, 0), (1, 5, 1), (3, 2, 8), (2, 3, 4), (6, 3, 3), (8, 2, 3), (1, 0, 3), (5, 6, 4), (0, 8, 0), (1, 0, 7), (5, 3, 8), (3, 3, 3), (6, 1, 2), (6, 2, 2), (5, 4, 3), (2, 1, 7), (8, 4, 0), (0, 3, 1), (6, 1, 4), (4, 3, 8), (8, 4, 6), (3, 5, 2), (7, 4, 8), (4, 8, 5), (8, 6, 7), (2, 7, 1), (8, 0, 5), (2, 8, 3), (8, 6, 3), (0, 5, 2), (5, 5, 2), (1, 7, 1), (2, 0, 0), (0, 2, 4), (6, 5, 2), (0, 2, 6), (4, 6, 0), (4, 1, 2), (5, 3, 1), (8, 5, 6), (0, 4, 1), (1, 2, 7), (5, 3, 5), (7, 6, 1), (0, 1, 1), (0, 8, 3), (5, 7, 8), (8, 7, 7), (7, 6, 5), (6, 2, 5), (5, 7, 4), (2, 7, 8), (4, 5, 5), (2, 8, 4), (4, 8, 6), (3, 8, 8), (4, 8, 4)}, 0: {(8, 6, 6), (0, 0, 8), (4, 7, 6), (0, 5, 7), (7, 2, 1), (3, 7, 0), (2, 5, 7), (2, 4, 8), (6, 6, 5), (6, 4, 8), (0, 1, 0), (5, 6, 2), (7, 8, 6), (5, 8, 3), (1, 0, 1), (7, 6, 4), (6, 2, 6), (6, 3, 7), (5, 6, 6), (8, 2, 7), (1, 6, 8), (1, 0, 5), (0, 0, 3), (4, 0, 4), (4, 5, 4), (6, 8, 0), (4, 4, 7), (2, 1, 3), (7, 1, 8), (4, 4, 5), (3, 5, 0), (4, 5, 8), (0, 3, 5), (4, 2, 1), (1, 1, 2), (2, 8, 7), (5, 2, 4), (8, 7, 8), (7, 2, 4), (8, 0, 7), (8, 6, 1), (5, 5, 0), (8, 3, 3), (3, 0, 6), (2, 7, 5), (3, 7, 1), (3, 8, 3), (2, 0, 2), (3, 8, 5), (6, 5, 0), (3, 2, 7), (0, 7, 3), (7, 6, 3), (3, 4, 4), (0, 4, 7), (1, 3, 4), (7, 8, 5), (8, 5, 2), (2, 3, 5), (5, 8, 2), (1, 5, 2), (1, 3, 0), (7, 8, 1), (5, 6, 5), (1, 0, 6), (5, 1, 6), (3, 3, 2), (2, 1, 8), (5, 4, 2), (6, 4, 3), (8, 7, 3), (6, 2, 1), (6, 1, 1), (2, 1, 4), (0, 3, 0), (0, 3, 6), (8, 1, 1), (7, 7, 4), (4, 2, 0), (1, 7, 8), (4, 4, 2)}}

all_combs = set()
for pct in range(9):
    assert(len(pct_to_combs[pct]) == (9 ** 2))
    all_combs |= pct_to_combs[pct]
    assert(len(all_combs) == ((9 ** 2) * (pct + 1)))
    all_nums, nums = set(), set()
    for s, c, l in pct_to_combs[pct]:
        key = s * 100 + c * 10 + l
        if key in nums:
            assert False
        nums.add(key)
        
    for s, c, l in all_combs:
        key = s * 100 + c * 10 + l
        if key in all_nums:
            assert False
        all_nums.add(key)

    plt.scatter(list(nums), np.ones(9 ** 2))
    plt.title("pct " + str(pct))
    plt.xlim(0, 888)
    plt.show()
    
    plt.scatter(list(all_nums), np.ones(len(all_combs)))
    plt.title("pcts up to " + str(pct))
    plt.xlim(0, 888)
    plt.show()
        
        

In [ ]:
plt.clf()
plt.plot(range(20),range(20))
plt.show()

In [ ]:

%matplotlib inline

In [ ]:
import random
import sys

def routine():
    shape_idx, col_idx, loc_idx, scale_idx = range(4)
    space_shape = (9, 9, 9, 9)

    all_combs = set([(s, c, l, sc) \
                     for s in range(space_shape[shape_idx]) \
                     for c in range(space_shape[col_idx]) \
                     for l in range(space_shape[loc_idx]) \
                     for sc in range(space_shape[scale_idx])
                    ])
    print(len(all_combs))

    pct_to_combs = { num:[] for num in range(max(space_shape))}

    pct = 0
    retry_ct = 0
    while pct < max(space_shape):
        assert(len(all_combs) == (max(space_shape) ** 4) - ((pct) * (max(space_shape) ** 3))), "bad"

        working_all_combs = all_combs.copy()
        if retry_ct % 25000 == 0:
            print("retry ct", retry_ct)

        try:
            shape_to_amt_needed = { num: 81 for num in range(space_shape[shape_idx]) }
            col_to_amt_needed = { num: 81 for num in range(space_shape[col_idx]) }
            loc_to_amt_needed = { num: 81 for num in range(space_shape[loc_idx]) }
            scale_to_amt_needed = { num: 81 for num in range(space_shape[scale_idx]) }

            combs_to_readd = set()
            while len(shape_to_amt_needed) \
                + len(col_to_amt_needed) \
                + len(loc_to_amt_needed) \
                + len(scale_to_amt_needed) \
                > 0:

    #             print(sum(shape_to_amt_needed.values()) \
    #                   + sum(col_to_amt_needed.values()) \
    #                   + sum(loc_to_amt_needed.values()) \
    #                   + sum(scale_to_amt_needed.values()))

                try:
                    s, c, l, sc = random.sample(working_all_combs, k=1)[0]
                except:
                    print(sys.exc_info()[0])
                    print(sum(shape_to_amt_needed.values()) \
                      + sum(col_to_amt_needed.values()) \
                      + sum(loc_to_amt_needed.values()) \
                      + sum(scale_to_amt_needed.values()))
                    assert False, "bleh"
                    continue

                print(s,c,l,sc)
                working_all_combs.remove((s, c, l, sc))
                assert (s, c, l, sc) not in pct_to_combs[pct], "not in"
                if s in shape_to_amt_needed and \
                   c in col_to_amt_needed and \
                   l in loc_to_amt_needed and \
                   sc in scale_to_amt_needed:
                    if shape_to_amt_needed[s] == 1:
                        del shape_to_amt_needed[s]
                    else:
                        shape_to_amt_needed[s] -= 1

                    if col_to_amt_needed[c] == 1:
                        del col_to_amt_needed[c]
                    else:
                        col_to_amt_needed[c] -= 1

                    if loc_to_amt_needed[l] == 1:
                        del loc_to_amt_needed[l]
                    else:
                        loc_to_amt_needed[l] -= 1

                    if scale_to_amt_needed[sc] == 1:
                        del scale_to_amt_needed[sc]
                    else:
                        scale_to_amt_needed[sc] -= 1
                    pct_to_combs[pct].append((s, c, l, sc))
                else:
                    combs_to_readd.add((s, c, l, sc))
            working_all_combs.update(combs_to_readd)
            print(len(pct_to_combs[pct]))
            assert len(pct_to_combs[pct]) == max(space_shape) ** 3 , "?"
            print(len(all_combs))
            assert len(all_combs) == (max(space_shape) ** 4) - ((pct + 1) * (max(space_shape) ** 3)) , "ok"
            pct += 1
            print("PCT: ", pct)
            all_combs = working_all_combs
        except AssertionError as e:
            print(e)
            
            retry_ct += 1
    return True, pct_to_combs

count = 0
while True:
    print("Retry ct: ", count)
    success, pct_to_combs = routine()
    if not success:
        print(pct_to_combs, " combs left")
        count += 1
        continue

for key, val in pct_to_combs.items():
    print("PCT:", key, "/", max(space_shape))
    print(val)
    print("\n")